In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch import nn
from torchvision import transforms, models

In [3]:
from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, RandomBrightnessContrast,
    Normalize, ShiftScaleRotate, CoarseDropout, ColorJitter, RandomGamma, GaussianBlur,
    OneOf
)
from albumentations.pytorch import ToTensorV2
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [8]:
# Устанавливаем необходимые библиотеки, если они не установлены
# !pip install albumentations --quiet
# !pip install timm --quiet

# Используем timm для удобного доступа к современным моделям
import timm

# Определение классов
class_names = ['cleaned', 'dirty']

# Корневой каталог данных
data_root = 'Data/plates'
print("Содержимое data_root:", os.listdir(data_root))



Содержимое data_root: ['.DS_Store', 'test', 'train']


In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from tqdm import tqdm
from torch import nn
from torchvision import models
from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, VerticalFlip, RandomBrightnessContrast,
    Normalize, ShiftScaleRotate, CoarseDropout, CenterCrop, OneOf,
    HueSaturationValue, ToGray, GaussNoise, MotionBlur,
    MedianBlur, Blur, CLAHE, RandomRotate90, Transpose, GridDistortion, OpticalDistortion,
    ElasticTransform
)
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from PIL import Image

# Установка необходимых библиотек, если они не установлены
# !pip install albumentations timm

import timm  # Библиотека с множеством современных моделей
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy

# Определение классов
class_names = ['cleaned', 'dirty']

# Корневой каталог данных
data_root = 'Data/plates'
print("Содержимое data_root:", os.listdir(data_root))

train_dir = 'drive/MyDrive/plates/train'
val_dir = 'drive/MyDrive/plates/val'

# Проверяем и создаем необходимые директории
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# Функция для копирования и разделения данных
def split_data(source_dir, train_dir, val_dir, split_ratio=0.9):
    files = os.listdir(source_dir)
    files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    np.random.shuffle(files)
    split_idx = int(len(files) * split_ratio)
    train_files = files[:split_idx]
    val_files = files[split_idx:]
    
    for file_name in train_files:
        src_file = os.path.join(source_dir, file_name)
        dest_file = os.path.join(train_dir, file_name)
        # Копируем только если файл не существует в целевой директории
        if not os.path.exists(dest_file):
            shutil.copy(src_file, dest_file)
    
    for file_name in val_files:
        src_file = os.path.join(source_dir, file_name)
        dest_file = os.path.join(val_dir, file_name)
        if not os.path.exists(dest_file):
            shutil.copy(src_file, dest_file)

# Разделяем данные на обучающую и валидационную выборки
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    split_data(source_dir, train_class_dir, val_class_dir, split_ratio=0.9)

# Продвинутые аугментации с Albumentations
def get_train_transforms():
    return Compose([
        RandomResizedCrop(380, 380),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=45, p=0.5),
        HueSaturationValue(p=0.5),
        RandomBrightnessContrast(p=0.5),
        RandomRotate90(p=0.5),
        OneOf([
            ElasticTransform(p=0.5),
            GridDistortion(p=0.5),
            OpticalDistortion(p=0.5),
        ], p=0.5),
        OneOf([
            GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            MotionBlur(p=0.5),
            MedianBlur(blur_limit=5, p=0.5),
            Blur(blur_limit=5, p=0.5),
        ], p=0.5),
        CoarseDropout(max_holes=8, max_height=40, max_width=40, fill_value=0, p=0.5),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_val_transforms():
    return Compose([
        CenterCrop(380, 380),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# Кастомный датасет для использования с Albumentations
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx):
        try:
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        except Exception as e:
            print(f"Ошибка загрузки изображения {self.image_paths[idx]}: {e}")
            image = np.zeros((380, 380, 3), dtype=np.uint8)
        label = self.labels[idx]
        if self.transforms:
            augmented = self.transforms(image=image)
            image = augmented['image']
        return image, label

    def __len__(self):
        return len(self.image_paths)

# Собираем пути к изображениям и метки
def get_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(class_dir, file_name))
                labels.append(label_idx)
    return image_paths, labels

train_image_paths, train_labels = get_image_paths_and_labels(train_dir)
val_image_paths, val_labels = get_image_paths_and_labels(val_dir)

print(f"Количество обучающих изображений: {len(train_image_paths)}")
print(f"Количество валидационных изображений: {len(val_image_paths)}")

# Создаем датасеты и загрузчики данных
batch_size = 8  # Увеличение batch_size при наличии ресурсов

train_dataset = CustomImageDataset(train_image_paths, train_labels, transforms=get_train_transforms())
val_dataset = CustomImageDataset(val_image_paths, val_labels, transforms=get_val_transforms())

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Используем модель EfficientNet-B4 из timm
model = timm.create_model('tf_efficientnet_b4_ns', pretrained=True, num_classes=len(class_names))

# Используем GPU, если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

# Используем Mixup для аугментации данных
mixup_fn = Mixup(
    mixup_alpha=0.2,
    cutmix_alpha=0.2,
    cutmix_minmax=None,
    prob=1.0,
    switch_prob=0.5,
    mode='batch',
    label_smoothing=0.1,
    num_classes=len(class_names)
)

# Используем Soft Target Cross Entropy Loss
loss_fn = SoftTargetCrossEntropy()

# Оптимизатор и планировщик
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Используем смешанную точность
scaler = torch.cuda.amp.GradScaler()

# Функция обучения
def train_model(model, loss_fn, optimizer, scheduler, num_epochs, device):
    best_acc = 0.0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_dataloader
            else:
                model.eval()
                dataloader = val_dataloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader, desc=phase):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                if phase == 'train':
                    inputs, labels = mixup_fn(inputs, labels)

                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast():
                        outputs = model(inputs)
                        loss = loss_fn(outputs, labels)

                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()

                running_loss += loss.item() * inputs.size(0)
                if phase == 'val':
                    preds = torch.argmax(outputs, dim=1)
                    running_corrects += torch.sum(preds == labels.argmax(dim=1))

            epoch_loss = running_loss / len(dataloader.dataset)
            if phase == 'val':
                epoch_acc = running_corrects.double() / len(dataloader.dataset)
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), 'best_model.pth')
            else:
                print(f'{phase} Loss: {epoch_loss:.4f}')

            if phase == 'train':
                scheduler.step()

        print()
    print(f'Best val Acc: {best_acc:.4f}')

num_epochs = 20
train_model(model, loss_fn, optimizer, scheduler, num_epochs, device)

# Загружаем лучшую модель
model.load_state_dict(torch.load('best_model.pth'))

# Подготовка тестовых данных
test_dir = os.path.join(data_root, 'test')
test_image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Количество тестовых изображений: {len(test_image_paths)}")

test_dataset = CustomImageDataset(test_image_paths, [0]*len(test_image_paths), transforms=get_val_transforms())
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Предсказания на тестовых данных с использованием TTA
model.eval()
test_predictions = []
for inputs, _ in tqdm(test_dataloader, desc='Test'):
    inputs = inputs.to(device)
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            probs = nn.functional.softmax(outputs, dim=1)[:, 1]  # Вероятность класса 'dirty'
    test_predictions.extend(probs.cpu().numpy())

# Создание DataFrame для сабмита
submission_df = pd.DataFrame({
    'id': [os.path.basename(path).replace('.jpg', '').replace('.jpeg', '').replace('.png', '') for path in test_image_paths],
    'label': ['dirty' if pred > 0.5 else 'cleaned' for pred in test_predictions]
})

# Сортировка и сохранение
submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранен.")


Содержимое data_root: ['test', 'train']
Количество обучающих изображений: 40
Количество валидационных изображений: 19


c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


cuda
Epoch 1/20
----------


train:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

from tqdm import tqdm
from torch import nn
from torchvision import models
from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, RandomBrightnessContrast,
    Normalize, ShiftScaleRotate, CoarseDropout
)
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from PIL import Image

# Убедитесь, что все необходимые библиотеки установлены
# !pip install albumentations efficientnet_pytorch

class_names = ['cleaned', 'dirty']

data_root = 'drive/MyDrive/plates'
print("Содержимое data_root:", os.listdir(data_root))

train_dir = 'drive/MyDrive/plates'
val_dir = 'val'

# Проверяем и создаем необходимые директории
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# Функция для копирования и разделения данных
def split_data(source_dir, train_dir, val_dir, split_ratio=0.85):
    files = os.listdir(source_dir)
    files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    np.random.shuffle(files)
    split_idx = int(len(files) * split_ratio)
    train_files = files[:split_idx]
    val_files = files[split_idx:]
    for file_name in train_files:
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(train_dir, file_name))
    for file_name in val_files:
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(val_dir, file_name))

# Разделяем данные на обучающую и валидационную выборки
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    split_data(source_dir, train_class_dir, val_class_dir, split_ratio=0.85)

# Продвинутые аугментации с Albumentations
def get_train_transforms():
    return Compose([
        RandomResizedCrop(224, 224),
        HorizontalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
        RandomBrightnessContrast(p=0.5),
        CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.5),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_val_transforms():
    return Compose([
        RandomResizedCrop(224, 224),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# Кастомный датасет для использования с Albumentations
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx):
        try:
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        except Exception as e:
            print(f"Ошибка загрузки изображения {self.image_paths[idx]}: {e}")
            # Возвращаем пустое изображение и метку 0 (можно изменить по необходимости)
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        label = self.labels[idx]
        if self.transforms:
            augmented = self.transforms(image=image)
            image = augmented['image']
        return image, label

    def __len__(self):
        return len(self.image_paths)

# Собираем пути к изображениям и метки
def get_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Фильтрация по типу файла
                image_paths.append(os.path.join(class_dir, file_name))
                labels.append(label_idx)
    return image_paths, labels

train_image_paths, train_labels = get_image_paths_and_labels(train_dir)
val_image_paths, val_labels = get_image_paths_and_labels(val_dir)

print(f"Количество обучающих изображений: {len(train_image_paths)}")
print(f"Количество валидационных изображений: {len(val_image_paths)}")

# Создаем датасеты и загрузчики данных
batch_size = 8  # Уменьшение batch_size для экономии памяти

train_dataset = CustomImageDataset(train_image_paths, train_labels, transforms=get_train_transforms())
val_dataset = CustomImageDataset(val_image_paths, val_labels, transforms=get_val_transforms())

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Используем модель EfficientNet-B0 (менее ресурсоемкая) из torchvision
model = models.efficientnet_b0(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(num_ftrs, len(class_names))
)

# Используем GPU, если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Используем взвешенную CrossEntropyLoss, если классы несбалансированы
from collections import Counter

counter = Counter(train_labels)
class_weights = 1. / torch.tensor([counter[i] for i in range(len(class_names))], dtype=torch.float)
class_weights = class_weights.to(device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Оптимизатор и планировщик
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Используем смешанную точность (Mixed Precision) для экономии памяти
scaler = torch.cuda.amp.GradScaler()

# Функция обучения с ранним прекращением
def train_model(model, loss_fn, optimizer, scheduler, num_epochs, device, patience=7):
    best_acc = 0.0
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_dataloader
            else:
                model.eval()
                dataloader = val_dataloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader, desc=phase):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast():
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = loss_fn(outputs, labels)

                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Шаг планировщика только для валидации
            if phase == 'val':
                scheduler.step(epoch_loss)

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    epochs_no_improve = 0
                    # Сохраняем лучшую модель
                    torch.save(model.state_dict(), 'best_model.pth')
                else:
                    epochs_no_improve += 1

                if epochs_no_improve >= patience:
                    print(f'Early stopping at epoch {epoch + 1}')
                    return
        print()
    print(f'Best val Acc: {best_acc:.4f}')

num_epochs = 30
train_model(model, loss_fn, optimizer, scheduler, num_epochs, device)

# Загружаем лучшую модель
model.load_state_dict(torch.load('best_model.pth'))

# Подготовка тестовых данных
test_dir = os.path.join(data_root, 'test')
test_image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Количество тестовых изображений: {len(test_image_paths)}")

test_dataset = CustomImageDataset(test_image_paths, [0]*len(test_image_paths), transforms=get_val_transforms())
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Предсказания на тестовых данных
model.eval()
test_predictions = []
for inputs, _ in tqdm(test_dataloader, desc='Test'):
    inputs = inputs.to(device)
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            probs = nn.functional.softmax(outputs, dim=1)[:, 1]  # Вероятность класса 'dirty'
    test_predictions.extend(probs.cpu().numpy())

# Создание DataFrame для сабмита
submission_df = pd.DataFrame({
    'id': [os.path.basename(path).replace('.jpg', '').replace('.jpeg', '').replace('.png', '') for path in test_image_paths],
    'label': ['dirty' if pred > 0.5 else 'cleaned' for pred in test_predictions]
})

# Сортировка и сохранение
submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранен.")

Содержимое data_root: ['cleaned', 'dirty', 'test', 'train', 'val']
Количество обучающих изображений: 40
Количество валидационных изображений: 25
Epoch 1/30
----------


train: 100%|██████████| 5/5 [00:00<00:00, 10.78it/s]


train Loss: 0.7188 Acc: 0.5250


val: 100%|██████████| 4/4 [00:00<00:00, 15.44it/s]


val Loss: 0.7263 Acc: 0.4800

Epoch 2/30
----------


...

train Loss: 0.3407 Acc: 0.9250


val: 100%|██████████| 4/4 [00:00<00:00, 51.95it/s]


val Loss: 0.1470 Acc: 1.0000
Early stopping at epoch 15
Количество тестовых изображений: 744


Test: 100%|██████████| 93/93 [00:01<00:00, 47.57it/s]

Файл submission.csv сохранен.


In [17]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16464 sha256=fb0b616673aeb070121066c1e9f6d99a767de45679cb0b53b9c00c026551b504
  Stored in directory: c:\users\alexx\appdata\local\pip\cache\wheels\03\3f\e9\911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [5]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from tqdm import tqdm
from torch import nn
from torchvision import models
from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, VerticalFlip, RandomBrightnessContrast,
    Normalize, ShiftScaleRotate, CoarseDropout, ColorJitter, GaussianBlur,
    GridDistortion, ElasticTransform, RandomGamma, HueSaturationValue, CLAHE, Resize, CenterCrop
)
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from PIL import Image

# Убедитесь, что все необходимые библиотеки установлены
# !pip install albumentations efficientnet_pytorch

class_names = ['cleaned', 'dirty']

data_root = 'drive/MyDrive/plates'
print("Содержимое data_root:", os.listdir(data_root))

train_dir = os.path.join(data_root, 'train')
val_dir = os.path.join(data_root, 'val')

# Проверяем и создаем необходимые директории
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# Функция для копирования и разделения данных
def split_data(source_dir, train_dir, val_dir, split_ratio=0.85):
    files = os.listdir(source_dir)
    files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    np.random.shuffle(files)
    split_idx = int(len(files) * split_ratio)
    train_files = files[:split_idx]
    val_files = files[split_idx:]
    for file_name in train_files:
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(train_dir, file_name))
    for file_name in val_files:
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(val_dir, file_name))

# Разделяем данные на обучающую и валидационную выборки
for class_name in class_names:
    source_dir = os.path.join(data_root, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    split_data(source_dir, train_class_dir, val_class_dir, split_ratio=0.85)

# Обновленные аугментации с Albumentations
def get_train_transforms():
    return Compose([
        RandomResizedCrop(224, 224, scale=(0.8, 1.0)),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.7),
        RandomBrightnessContrast(p=0.7),
        ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
        GaussianBlur(blur_limit=(3,5), p=0.5),
        RandomGamma(p=0.5),
        HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        CLAHE(clip_limit=4.0, p=0.5),
        GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
        CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.5),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_val_transforms():
    return Compose([
        Resize(256, 256),
        CenterCrop(224, 224),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# Кастомный датасет для использования с Albumentations
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx):
        try:
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        except Exception as e:
            print(f"Ошибка загрузки изображения {self.image_paths[idx]}: {e}")
            # Возвращаем пустое изображение и метку 0
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        label = self.labels[idx]
        if self.transforms:
            augmented = self.transforms(image=image)
            image = augmented['image']
        return image, label

    def __len__(self):
        return len(self.image_paths)

# Собираем пути к изображениям и метки
def get_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(class_dir, file_name))
                labels.append(label_idx)
    return image_paths, labels

train_image_paths, train_labels = get_image_paths_and_labels(train_dir)
val_image_paths, val_labels = get_image_paths_and_labels(val_dir)

print(f"Количество обучающих изображений: {len(train_image_paths)}")
print(f"Количество валидационных изображений: {len(val_image_paths)}")

# Создаем датасеты и загрузчики данных
batch_size = 8  # Можно настроить в зависимости от доступной памяти

train_dataset = CustomImageDataset(train_image_paths, train_labels, transforms=get_train_transforms())
val_dataset = CustomImageDataset(val_image_paths, val_labels, transforms=get_val_transforms())

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Используем модель EfficientNet-B0
model = models.efficientnet_b0(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(num_ftrs, len(class_names))
)

# Используем GPU, если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Используем взвешенную CrossEntropyLoss
from collections import Counter

counter = Counter(train_labels)
class_weights = 1. / torch.tensor([counter[i] for i in range(len(class_names))], dtype=torch.float)
class_weights = class_weights.to(device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Оптимизатор и планировщик
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Используем смешанную точность
scaler = torch.cuda.amp.GradScaler()

# Функция обучения с ранним прекращением
def train_model(model, loss_fn, optimizer, scheduler, num_epochs, device, patience=7):
    best_acc = 0.0
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_dataloader
            else:
                model.eval()
                dataloader = val_dataloader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloader, desc=phase):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast():
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = loss_fn(outputs, labels)

                    if phase == 'train':
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()

                if phase == 'train':
                    scheduler.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Проверка наилучшей точности
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    epochs_no_improve = 0
                    # Сохраняем лучшую модель
                    torch.save(model.state_dict(), 'best_model.pth')
                else:
                    epochs_no_improve += 1

                if epochs_no_improve >= patience:
                    print(f'Early stopping at epoch {epoch + 1}')
                    print(f'Best val Acc: {best_acc:.4f}')
                    return
        print()
    print(f'Best val Acc: {best_acc:.4f}')

num_epochs = 50
train_model(model, loss_fn, optimizer, scheduler, num_epochs, device, patience=7)

# Загружаем лучшую модель
model.load_state_dict(torch.load('best_model.pth'))

# Подготовка тестовых данных
test_dir = os.path.join(data_root, 'test')
test_image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Количество тестовых изображений: {len(test_image_paths)}")

# Используем те же трансформации, что и для валидации
test_dataset = CustomImageDataset(test_image_paths, [0]*len(test_image_paths), transforms=get_val_transforms())
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=0)

# Используем TTA для предсказаний
def get_tta_transforms():
    tta_transforms = []
    flips = [None, HorizontalFlip(p=1.0), VerticalFlip(p=1.0)]
    for flip in flips:
        transforms = [Resize(256, 256), CenterCrop(224, 224)]
        if flip:
            transforms.append(flip)
        transforms.extend([
            Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        tta_transforms.append(Compose(transforms))
    return tta_transforms

tta_transforms = get_tta_transforms()

# Предсказания на тестовых данных с TTA
model.eval()
test_predictions = []
for idx in tqdm(range(len(test_dataset)), desc='Test'):
    images = []
    original_image = np.array(Image.open(test_image_paths[idx]).convert("RGB"))
    for tta_transform in tta_transforms:
        augmented = tta_transform(image=original_image)
        image = augmented['image']
        images.append(image)
    images = torch.stack(images).to(device)
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            outputs = model(images)
            outputs = nn.functional.softmax(outputs, dim=1)[:, 1]  # Вероятность класса 'dirty'
            probs = outputs.mean().item()
    test_predictions.append(probs)

# Создание DataFrame для сабмита
submission_df = pd.DataFrame({
    'id': [os.path.basename(path).replace('.jpg', '').replace('.jpeg', '').replace('.png', '') for path in test_image_paths],
    'label': ['dirty' if pred > 0.5 else 'cleaned' for pred in test_predictions]
})

# Сортировка и сохранение
submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранен.")


Содержимое data_root: ['cleaned', 'dirty', 'test', 'train', 'val']
Количество обучающих изображений: 40
Количество валидационных изображений: 24


C:\Users\alexx\AppData\Local\Temp\ipykernel_8316\901042687.py:71: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),


Epoch 1/50
----------


train: 100%|██████████| 5/5 [00:00<00:00,  8.43it/s]


train Loss: 0.7309 Acc: 0.5500


...

train Loss: 0.0428 Acc: 1.0000


val: 100%|██████████| 3/3 [00:00<00:00, 55.56it/s]


val Loss: 0.2753 Acc: 0.9583
Early stopping at epoch 16
Best val Acc: 1.0000
Количество тестовых изображений: 744


Test: 100%|██████████| 744/744 [00:07<00:00, 97.27it/s] 

Файл submission.csv сохранен.


In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from tqdm import tqdm
from torch import nn
from torchvision import models
from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, VerticalFlip, ShiftScaleRotate,
    RandomBrightnessContrast, Normalize, CoarseDropout, ColorJitter, GaussianBlur,
    GridDistortion, ElasticTransform, RandomGamma, HueSaturationValue, CLAHE,
    Resize, CenterCrop, CoarseDropout, OneOf, OpticalDistortion, MotionBlur, MedianBlur,
    RandomRain, RandomSnow, RandomFog, RandomShadow, RandomSunFlare, Perspective
)
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset
from tqdm import tqdm
from torch import nn
from torchvision import models
from albumentations import (
    Compose, RandomResizedCrop, HorizontalFlip, VerticalFlip, ShiftScaleRotate,
    RandomBrightnessContrast, Normalize, CoarseDropout, ColorJitter, GaussianBlur,
    GridDistortion, ElasticTransform, RandomGamma, HueSaturationValue, CLAHE,
    Resize, CenterCrop, OneOf, OpticalDistortion, MotionBlur, MedianBlur,
    RandomRain, RandomSnow, RandomFog, RandomShadow, RandomSunFlare, Perspective
)
from albumentations.pytorch import ToTensorV2
from PIL import Image
import random

# Убедитесь, что все необходимые библиотеки установлены
# !pip install albumentations

# Определяем классы
class_names = ['cleaned', 'dirty']

# Путь к данным
data_root = 'drive/MyDrive/plates'
print("Содержимое data_root:", os.listdir(data_root))

# Директория с обучающими данными
train_dir = os.path.join(data_root, 'train')

# Проверяем и создаем необходимые директории
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)

# Функция для копирования данных в обучающую директорию
def prepare_data(source_dir, target_dir):
    for class_name in class_names:
        class_source_dir = os.path.join(source_dir, class_name)
        class_target_dir = os.path.join(target_dir, class_name)
        os.makedirs(class_target_dir, exist_ok=True)
        files = os.listdir(class_source_dir)
        files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        for file_name in files:
            shutil.copy(os.path.join(class_source_dir, file_name), os.path.join(class_target_dir, file_name))

# Копируем все данные в обучающую директорию
prepare_data(data_root, train_dir)

# Обновленные аугментации с Albumentations
def get_train_transforms():
    return Compose([
        RandomResizedCrop(224, 224, scale=(0.8, 1.0)),
        OneOf([
            HorizontalFlip(p=1.0),
            VerticalFlip(p=1.0),
        ], p=0.5),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.7),
        OneOf([
            RandomBrightnessContrast(p=1.0),
            ColorJitter(p=1.0),
        ], p=0.7),
        OneOf([
            GaussianBlur(p=1.0),
            MotionBlur(p=1.0),
            MedianBlur(p=1.0),
        ], p=0.5),
        OneOf([
            RandomRain(p=1.0),
            RandomSnow(p=1.0),
            RandomFog(p=1.0),
            RandomShadow(p=1.0),
            RandomSunFlare(p=1.0),
        ], p=0.3),
        Perspective(distortion_scale=0.5, p=0.5),
        CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.5),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_test_transforms():
    return Compose([
        Resize(256, 256),
        CenterCrop(224, 224),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# Кастомный датасет для использования с Albumentations
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx):
        try:
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        except Exception as e:
            print(f"Ошибка загрузки изображения {self.image_paths[idx]}: {e}")
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        label = self.labels[idx]
        if self.transforms:
            augmented = self.transforms(image=image)
            image = augmented['image']
        return image, label

    def __len__(self):
        return len(self.image_paths)

# Собираем пути к изображениям и метки
def get_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(class_dir, file_name))
                labels.append(label_idx)
    return image_paths, labels

train_image_paths, train_labels = get_image_paths_and_labels(train_dir)

print(f"Количество обучающих изображений: {len(train_image_paths)}")

# Создаем датасет и загрузчик данных
batch_size = 8  # Настройте в зависимости от доступной памяти

train_dataset = CustomImageDataset(train_image_paths, train_labels, transforms=get_train_transforms())
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Используем модель EfficientNet-B4
model = models.efficientnet_b4(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(num_ftrs, len(class_names))
)

# Размораживаем слои для тонкой настройки
for param in model.parameters():
    param.requires_grad = True

# Используем GPU, если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Используем Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean', weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.weight = weight

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(weight=self.weight, reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * ((1 - pt) ** self.gamma) * ce_loss
        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        else:
            return focal_loss

# Используем взвешенную Focal Loss
from collections import Counter

counter = Counter(train_labels)
class_weights = 1. / torch.tensor([counter[i] for i in range(len(class_names))], dtype=torch.float)
class_weights = class_weights.to(device)

loss_fn = FocalLoss(weight=class_weights)

# Оптимизатор и планировщик
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Функция обучения
def train_model(model, loss_fn, optimizer, scheduler, num_epochs, device):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_dataloader, desc='Train'):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        scheduler.step()

        epoch_loss = running_loss / len(train_dataloader.dataset)
        epoch_acc = running_corrects.double() / len(train_dataloader.dataset)

        print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # Сохраняем модель после обучения
    torch.save(model.state_dict(), 'best_model.pth')
    print('Обучение завершено и модель сохранена.')

num_epochs = 100  # Вы можете увеличить количество эпох при необходимости
train_model(model, loss_fn, optimizer, scheduler, num_epochs, device)

# Загружаем обученную модель
model.load_state_dict(torch.load('best_model.pth'))

# Подготовка тестовых данных
test_dir = os.path.join(data_root, 'test')
test_image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir)
                    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Количество тестовых изображений: {len(test_image_paths)}")

# Создаем датасет и загрузчик для тестовых данных
test_dataset = CustomImageDataset(test_image_paths, [0]*len(test_image_paths), transforms=get_test_transforms())
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=0
)

# Используем TTA для предсказаний
def get_tta_transforms():
    tta_transforms = []
    flips = [None, HorizontalFlip(p=1.0), VerticalFlip(p=1.0)]
    for flip in flips:
        transforms = [Resize(256, 256), CenterCrop(224, 224)]
        if flip:
            transforms.append(flip)
        transforms.extend([
            Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        tta_transforms.append(Compose(transforms))
    return tta_transforms

tta_transforms = get_tta_transforms()

# Предсказания на тестовых данных с TTA
model.eval()
test_predictions = []
for idx in tqdm(range(len(test_dataset)), desc='Test'):
    images = []
    original_image = np.array(Image.open(test_image_paths[idx]).convert("RGB"))
    for tta_transform in tta_transforms:
        augmented = tta_transform(image=original_image)
        image = augmented['image']
        images.append(image)
    images = torch.stack(images).to(device)
    with torch.no_grad():
        outputs = model(images)
        outputs = nn.functional.softmax(outputs, dim=1)[:, 1]  # Вероятность класса 'dirty'
        probs = outputs.mean().item()
    test_predictions.append(probs)

# Создание DataFrame для сабмита
submission_df = pd.DataFrame({
    'id': [os.path.basename(path).split('.')[0] for path in test_image_paths],
    'label': ['dirty' if pred > 0.5 else 'cleaned' for pred in test_predictions]
})

# Сортировка и сохранение
submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранен.")


Содержимое data_root: ['cleaned', 'dirty', 'test', 'train', 'val']
Количество обучающих изображений: 40


C:\Users\alexx\AppData\Local\Temp\ipykernel_18048\4051932503.py:77: UserWarning: Argument 'distortion_scale' is not valid and will be ignored.
  Perspective(distortion_scale=0.5, p=0.5),
c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alexx\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100
----------


Train: 100%|██████████| 5/5 [00:01<00:00,  4.66it/s]


Loss: 0.0000 Acc: 0.5250
Epoch 2/100
----------


...

Loss: 0.0000 Acc: 0.8000
Epoch 100/100
----------


Train: 100%|██████████| 5/5 [00:00<00:00,  9.37it/s]


Loss: 0.0000 Acc: 0.9250
Обучение завершено и модель сохранена.
Количество тестовых изображений: 744


Test: 100%|██████████| 744/744 [00:11<00:00, 67.20it/s]

Файл submission.csv сохранен.


In [8]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset
from tqdm import tqdm
from torch import nn
from torchvision import models
from sklearn.model_selection import StratifiedKFold
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip, ColorJitter, Normalize,
    Resize, ToGray, Perspective
)
from albumentations.pytorch import ToTensorV2
from PIL import Image
import cv2

# Определяем классы
class_names = ['cleaned', 'dirty']

# Путь к данным
data_root = 'drive/MyDrive/plates'
print("Содержимое data_root:", os.listdir(data_root))

# Собираем пути к изображениям и метки
def get_image_paths_and_labels(data_root):
    image_paths = []
    labels = []
    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_root, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(class_dir, file_name))
                labels.append(label_idx)
    return image_paths, labels

image_paths, labels = get_image_paths_and_labels(data_root)
print(f"Всего изображений: {len(image_paths)}")


Содержимое data_root: ['cleaned', 'dirty', 'test', 'train', 'val']
Всего изображений: 40


In [9]:
def get_train_transforms():
    return Compose([
        Resize(256, 256),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ColorJitter(p=0.5),
        ToGray(p=0.2),
        Perspective(p=0.5),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_val_transforms():
    return Compose([
        Resize(256, 256),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])


In [10]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]
        if self.transforms:
            augmented = self.transforms(image=image)
            image = augmented['image']
        return image, label

    def __len__(self):
        return len(self.image_paths)


In [3]:
# Install necessary libraries
# Uncomment the following lines if you're running in a new environment
# !pip install albumentations
# !pip install efficientnet_pytorch
# !pip install timm

import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch import nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from albumentations import (
    Compose, HorizontalFlip, VerticalFlip, ColorJitter, ToGray, Perspective, Normalize,
    ShiftScaleRotate, RandomBrightnessContrast, CoarseDropout
)
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform
import albumentations as A
import cv2
from tqdm import tqdm
from PIL import Image
import random
import copy
import timm
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set the random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Define the class names
class_names = ['cleaned', 'dirty']

# Define the data root directory
data_root = 'drive/MyDrive/plates'  # Replace with your actual data directory
print("Contents of data_root:", os.listdir(data_root))

# Image size for EfficientNet-B4
image_size = 380

# Define transformations using Albumentations
def get_train_transforms():
    return Compose([
        # We remove Resize here because we'll resize images before transformations
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        RandomBrightnessContrast(p=0.5),
        ColorJitter(p=0.5),
        ToGray(p=0.1),
        Perspective(p=0.5),
        CoarseDropout(max_height=32, max_width=32, max_holes=5, p=0.5),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

def get_val_transforms():
    return Compose([
        # We remove Resize here because we'll resize images before transformations
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None, use_mixup=False, use_cutmix=False):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms
        self.use_mixup = use_mixup
        self.use_cutmix = use_cutmix

    def __getitem__(self, idx):
        # Load and resize the primary image
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (image_size, image_size))
        label = self.labels[idx]

        if self.use_mixup or self.use_cutmix:
            # Load and resize the secondary image
            idx2 = random.randint(0, len(self.image_paths) - 1)
            image2 = cv2.imread(self.image_paths[idx2])
            image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
            image2 = cv2.resize(image2, (image_size, image_size))
            label2 = self.labels[idx2]

            if self.use_cutmix:
                # Apply CutMix
                lam = np.random.beta(1.0, 1.0)
                bbx1, bby1, bbx2, bby2 = self.rand_bbox(image.shape[0], image.shape[1], lam)
                image[bbx1:bbx2, bby1:bby2, :] = image2[bbx1:bbx2, bby1:bby2, :]
                label = label * lam + label2 * (1. - lam)
            elif self.use_mixup:
                # Apply Mixup
                lam = np.random.beta(1.0, 1.0)
                image = (image * lam + image2 * (1 - lam)).astype(np.uint8)
                label = label * lam + label2 * (1. - lam)

            if self.transforms:
                augmented = self.transforms(image=image)
                image = augmented['image']
        else:
            if self.transforms:
                augmented = self.transforms(image=image)
                image = augmented['image']

        return image, label

    def __len__(self):
        return len(self.image_paths)

    def rand_bbox(self, H, W, lam):
        cut_rat = np.sqrt(1. - lam)
        cut_w = int(W * cut_rat)
        cut_h = int(H * cut_rat)

        # uniform
        cx = np.random.randint(W)
        cy = np.random.randint(H)

        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)

        return bbx1, bby1, bbx2, bby2

# Function to get image paths and labels
def get_image_paths_and_labels(data_root):
    image_paths = []
    labels = []
    for label_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(data_root, class_name)
        for file_name in os.listdir(class_dir):
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(class_dir, file_name))
                labels.append(label_idx)
    return image_paths, labels

# Prepare the dataset
image_paths, labels = get_image_paths_and_labels(data_root)
print(f"Total images: {len(image_paths)}")

# Stratified K-Fold Cross Validation
num_epochs = 100
batch_size = 16
k_folds = 2
use_mixup = True
use_cutmix = True

skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=seed)
models_list = []

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f'Fold {fold + 1}/{k_folds}')
    print('-' * 10)

    # Create datasets for the current fold
    train_dataset = CustomImageDataset(
        [image_paths[i] for i in train_idx],
        [labels[i] for i in train_idx],
        transforms=get_train_transforms(),
        use_mixup=use_mixup,
        use_cutmix=use_cutmix
    )
    val_dataset = CustomImageDataset(
        [image_paths[i] for i in val_idx],
        [labels[i] for i in val_idx],
        transforms=get_val_transforms(),
        use_mixup=False,
        use_cutmix=False  # We don't use Mixup/CutMix in validation
    )

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    # Initialize the model
    model = EfficientNet.from_pretrained('efficientnet-b4')
    num_ftrs = model._fc.in_features
    model._fc = nn.Linear(num_ftrs, 1)  # For BCEWithLogitsLoss
    model = model.to(device)

    # Define loss function and optimizer
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)

    # Training loop
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    early_stop_patience = 15

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Train phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels_batch in tqdm(train_loader, desc='Training'):
            inputs = inputs.to(device)
            labels_batch = labels_batch.to(device).float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels_batch)
            preds = torch.sigmoid(outputs) > 0.5
            loss.backward()
            optimizer.step()
            scheduler.step(epoch + len(train_loader) / len(train_loader))

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels_batch.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        all_labels = []
        all_preds = []

        with torch.no_grad():
            for inputs, labels_batch in tqdm(val_loader, desc='Validation'):
                inputs = inputs.to(device)
                labels_batch = labels_batch.to(device).float().unsqueeze(1)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels_batch)
                preds = torch.sigmoid(outputs) > 0.5

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels_batch.data)

                all_labels.extend(labels_batch.cpu().numpy())
                all_preds.extend(torch.sigmoid(outputs).cpu().numpy())

        val_loss_epoch = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Val Loss: {val_loss_epoch:.4f} Acc: {val_acc:.4f}')

        # Calculate additional metrics
        all_labels_np = np.array(all_labels)
        all_preds_np = np.array(all_preds)
        val_precision = precision_score(all_labels_np, all_preds_np > 0.5)
        val_recall = recall_score(all_labels_np, all_preds_np > 0.5)
        val_f1 = f1_score(all_labels_np, all_preds_np > 0.5)
        print(f'Val Precision: {val_precision:.4f} Recall: {val_recall:.4f} F1: {val_f1:.4f}')

        # Check for improvement
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            patience_counter = 0
            print('Improved validation accuracy. Saving model...')
        else:
            patience_counter += 1
            if patience_counter >= early_stop_patience:
                print('Early stopping due to no improvement in validation accuracy.')
                break

    print(f'Best Val Acc for fold {fold + 1}: {best_acc:.4f}')
    # Load best model weights
    model.load_state_dict(best_model_wts)
    # Save the best model for this fold
    fold_model_path = f'best_model_fold_{fold}.pth'
    torch.save(model.state_dict(), fold_model_path)
    models_list.append(fold_model_path)

# Prepare test data
test_dir = os.path.join(data_root, 'test')
test_image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir)
                    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Number of test images: {len(test_image_paths)}")

# Test-Time Augmentation (TTA)
def tta_inference(model, image):
    tta_transforms = []
    tta_transforms.append(get_val_transforms())
    tta_transforms.append(Compose([A.HorizontalFlip(p=1.0)] + get_val_transforms().transforms))
    tta_transforms.append(Compose([A.VerticalFlip(p=1.0)] + get_val_transforms().transforms))
    tta_transforms.append(Compose([A.Transpose(p=1.0)] + get_val_transforms().transforms))

    preds = []
    for transformer in tta_transforms:
        img = cv2.resize(image, (image_size, image_size))
        augmented = transformer(image=img)
        img = augmented['image'].unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(img)
            prob = torch.sigmoid(output).cpu().numpy()
            preds.append(prob[0][0])
    return np.mean(preds)

# Ensemble predictions
ensemble_preds = np.zeros(len(test_image_paths))

for model_path in models_list:
    model = EfficientNet.from_pretrained('efficientnet-b4')
    num_ftrs = model._fc.in_features
    model._fc = nn.Linear(num_ftrs, 1)
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()

    preds = []
    for idx, img_path in enumerate(tqdm(test_image_paths, desc=f'Predicting with {model_path}')):
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pred = tta_inference(model, image)
        preds.append(pred)

    ensemble_preds += np.array(preds)

# Average the predictions
ensemble_preds /= len(models_list)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': [os.path.basename(path).split('.')[0] for path in test_image_paths],
    'label': ['dirty' if pred > 0.5 else 'cleaned' for pred in ensemble_preds]
})

# Sort and save the submission
submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("Saved submission.csv")

# Optionally, you can print the final submission DataFrame
print(submission_df.head())


Using device: cuda
Contents of data_root: ['cleaned', 'dirty', 'test', 'train', 'val']
Total images: 40
Fold 1/2
----------
Loaded pretrained weights for efficientnet-b4
Epoch 1/100
----------


Training: 100%|██████████| 2/2 [00:10<00:00,  5.08s/it]


Train Loss: 0.7093 Acc: 0.2500


Validation: 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


Val Loss: 0.6793 Acc: 0.5500
Val Precision: 1.0000 Recall: 0.1000 F1: 0.1818
Improved validation accuracy. Saving model...
Epoch 2/100
----------


...

Val Loss: 0.7425 Acc: 0.7500
Val Precision: 0.7273 Recall: 0.8000 F1: 0.7619
Early stopping due to no improvement in validation accuracy.
Best Val Acc for fold 2: 0.8000
Number of test images: 744
Loaded pretrained weights for efficientnet-b4


Predicting with best_model_fold_0.pth: 100%|██████████| 744/744 [08:47<00:00,  1.41it/s]


Loaded pretrained weights for efficientnet-b4


Predicting with best_model_fold_1.pth: 100%|██████████| 744/744 [08:39<00:00,  1.43it/s]

Saved submission.csv
     id    label
0  0000    dirty
1  0001    dirty
2  0002    dirty
3  0003    dirty
4  0004  cleaned
